In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
# from ddpg_torch import Agent
# import gym
# import numpy as np
# import torch

# env = gym.make("LunarLanderContinuous-v2")
# agent = Agent(alpha=0.000025, beta=0.00025, input_dims=[4], tau=0.001, env=env, batch_size=64, layer1_size=400, layer2_size=300, n_actions=2)

# np.random.seed(0)

# score_history = []
# for i in range(1000):
#     done = False
#     score = 0
#     observation, info = env.reset()
#     while not done:
#         action = agent.choose_action(observation)
#         new_state, reward, terminated, truncated, step_info = env.step(action)
#         done = terminated or truncated
#         agent.remember(observation, action, reward, new_state, int(done))
#         agent.learn()
#         score += reward
#         observation = new_state
    
#     score_history.append(score)
#     print('episode ', i, 'score %.2f' % score
#           , '100 game average %.2f' % np.mean(score_history[-100:]))
    
#     if i % 25 == 0:
#         agent.save_models()
    
#     filename = 'scores.png'
#     x = [i+1 for i in range(len(score_history))]
#     running_avg = np.zeros(len(score_history))

#     import numpy as np
# import matplotlib.pyplot as plt

# # after the loop, once score_history is filled:
# x = np.arange(1, len(score_history)+1)
# running_avg = np.zeros_like(x, dtype=float)

# for t in range(len(x)):
#     running_avg[t] = np.mean(score_history[max(0, t-99):t+1])

# plt.plot(x, score_history, label='Score per episode')
# plt.plot(x, running_avg, label='100-episode running average')
# plt.xlabel('Episode')
# plt.ylabel('Score')
# plt.legend()
# plt.show()

In [0]:
# import os
# os.makedirs('tmp/ddpg', exist_ok=True)

# # Gym’s checker and NumPy ≥1.25
# import numpy as np
# np.bool8 = np.bool_

# import gymnasium as gym
# import torch
# import matplotlib.pyplot as plt
# from ddpg_torch import Agent

# # create continuous-action environment
# env = gym.make("LunarLanderContinuous-v3")

# # pull dims straight from env
# obs_dim    = env.observation_space.shape[0]   # 8
# n_actions  = env.action_space.shape[0]        # 2

# agent = Agent(
#     alpha=2.5e-05, beta=2.5e-04,
#     input_dims=[obs_dim], tau=0.001, env=env,
#     batch_size=64, layer1_size=400, layer2_size=300,
#     n_actions=n_actions
# )

# # reproducibility
# np.random.seed(0)
# torch.manual_seed(0)
# env.reset(seed=0)

# score_history = []
# for i in range(1000):
#     observation, _ = env.reset()
#     done = False
#     score = 0.0

#     while not done:
#         # now we can directly use the continuous output
#         action = agent.choose_action(observation)  

#         new_state, reward, terminated, truncated, _ = env.step(action)
#         done = terminated or truncated

#         agent.remember(observation, action, reward, new_state, int(done))
#         agent.learn()

#         score += reward
#         observation = new_state

#     score_history.append(score)
#     avg100 = np.mean(score_history[-100:])
#     print(f"Episode {i:4d}  Score {score:6.2f}  100-avg {avg100:6.2f}")

#     if i % 25 == 0:
#         agent.save_models()

# # plot results
# x = np.arange(1, len(score_history) + 1)
# running_avg = np.zeros_like(x, dtype=float)
# for t in range(len(x)):
#     running_avg[t] = np.mean(score_history[max(0, t-99):t+1])

# plt.plot(x, score_history, label="Score per episode")
# plt.plot(x, running_avg, label="100-episode running average")
# plt.xlabel("Episode")
# plt.ylabel("Score")
# plt.legend()
# plt.show()


In [0]:
%pip install swig
%pip install gymnasium[box2d]

In [0]:
%pip install git+https://github.com/Farama-Foundation/Gymnasium-Robotics.git
%pip uninstall -y mujoco-py
%pip install mujoco
%pip install gymnasium[robotics]

In [0]:
dbutils.library.restartPython()

In [0]:
import gymnasium as gym
import gymnasium_robotics

# 1) Create the FetchPush-v4 environment
env = gym.make("FetchPush-v4")

# 2) Reset the environment to get the initial observation
obs, info = env.reset()

# 3) Print the observation structure
print("Observation keys:", obs.keys())
print("Sample observation:")
for key, value in obs.items():
    print(f"  {key}: shape={value.shape}, type={type(value)}")


In [0]:
import gymnasium as gym
import gymnasium_robotics  # Ensures robotics envs are registered

# Print all Fetch-related environments
fetch_envs = [env_spec.id for env_spec in gym.registry.values() if "Fetch" in env_spec.id]
print(fetch_envs)

In [0]:
import os
os.makedirs('tmp/ddpg', exist_ok=True)

# Gym’s PassiveEnvChecker fix
import numpy as np
np.bool8 = np.bool_

import gymnasium as gym
import gymnasium_robotics  # ensures the Fetch* envs are registered
import torch
import matplotlib.pyplot as plt

from ddpg_torch import Agent

# 1) Create the FetchPush-v4 env
env = gym.make("FetchPush-v4")

# 2) Combine 'observation' and 'desired_goal' as input
obs_space = env.observation_space.spaces['observation']
goal_space = env.observation_space.spaces['desired_goal']
obs_dim = obs_space.shape[0] + goal_space.shape[0]
act_dim = env.action_space.shape[0]

# 3) Build your DDPG agent
agent = Agent(
    alpha=1e-4, beta=1e-3,  # ↑ Increased LR for better early learning
    input_dims=[obs_dim], tau=0.001, env=env,
    batch_size=64, layer1_size=400, layer2_size=300,
    n_actions=act_dim
)

# reproducibility
np.random.seed(0)
torch.manual_seed(0)
env.reset(seed=0)

# 4) Training loop
score_history = []
success_history = []

update_freq = 2
warmup_episodes = 10
warmup_steps = 1000
total_steps = 0

for i in range(1000):
    obs_dict, _ = env.reset()
    agent.noise.reset()
    state = np.concatenate([obs_dict['observation'], obs_dict['desired_goal']])
    done = False
    score = 0.0
    step = 0
    success = 0

    while not done:
        # Warm-up exploration (random action)
        if i < warmup_episodes:
            action = env.action_space.sample()
        else:
            action = agent.choose_action(state)
            action += 0.1 * np.random.randn(act_dim)  # Gaussian noise
            action = np.clip(action, -1.0, 1.0)

        next_obs_dict, _, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        # HER-style reward recomputation
        achieved_goal = next_obs_dict["achieved_goal"]
        desired_goal = next_obs_dict["desired_goal"]
        #reward = env.compute_reward(achieved_goal, desired_goal, info={})
        reward = agent._reward_env.compute_reward(achieved_goal, desired_goal, info={})

        next_state = np.concatenate([next_obs_dict['observation'], desired_goal])
        agent.remember(state, action, reward, next_state, int(done))

        # Learn only after sufficient buffer and at intervals
        if total_steps > warmup_steps and step % update_freq == 0:
            agent.learn()

        score += reward
        success += info.get("is_success", 0.0)
        state = next_state
        step += 1
        total_steps += 1

    score_history.append(score)
    success_history.append(success)
    avg_score = np.mean(score_history[-100:])
    avg_success = np.mean(success_history[-100:]) * 100

    print(f"Episode {i:4d}  Score {score:6.2f}  100-avg {avg_score:6.2f}  Success {avg_success:.1f}%")

    if i % 25 == 0:
        agent.save_models()

# 5) Plot results
x = np.arange(1, len(score_history) + 1)
running_avg = np.zeros_like(x, dtype=float)
for t in range(len(x)):
    running_avg[t] = np.mean(score_history[max(0, t-99):t+1])

plt.figure(figsize=(12, 5))
plt.plot(x, score_history, label="Score per episode")
plt.plot(x, running_avg, label="100-episode running avg")
plt.xlabel("Episode")
plt.ylabel("Score")
plt.title("DDPG on FetchPush-v4")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()
